### Importing Libraries

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pickle

### Dynamic Gesture Collection

In [2]:
# Setup MediaPipe instance
# label_list = [15] * 20 # 15 is the label for the gesture
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Start capturing video
cap = cv2.VideoCapture(0)
all_gestures = []

if not cap.isOpened():
    print("Error opening video stream or file")

local_landmarks_data = []
world_landmarks_data = []
recording = False

while True:
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue  # Skip the rest of the loop if frame capture failed

    # Convert the image color from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = hands.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # Process local landmarks for drawing
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Store local landmarks if recording
            if recording:
                # Store local landmarks
                frame_landmarks = []
                for lm in hand_landmarks.landmark:
                    # Convert coordinates to relative coordinates (based on image dimensions)
                    frame_landmarks.append((lm.x, lm.y, lm.z))
                local_landmarks_data.append(frame_landmarks)

            # Collect world landmarks data for storage
            if results.multi_hand_world_landmarks:
                frame_world_landmarks = []
                for hand_world_landmarks in results.multi_hand_world_landmarks:
                    # Flatten the list of landmarks
                    landmarks = [(lm.x, lm.y, lm.z) for lm in hand_world_landmarks.landmark]
                    frame_world_landmarks.extend(landmarks)  # Use extend instead of append
                world_landmarks_data.append(frame_world_landmarks)

    # Display the image
    if recording:
        cv2.putText(image, "Recording Gesture", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('MediaPipe Hands', image)
    
    key = cv2.waitKey(1)
    if key == ord('s'):  # Start recording gesture
        recording = True
        local_landmarks_data = []
        world_landmarks_data = []
    elif key == ord('w'):  # End recording gesture
        recording = False
        all_gestures.append(world_landmarks_data)
        print(f"Gesture recorded. Total recorded gestures: {len(all_gestures)}")
    elif key == ord('q'):  # Quit
        break

# Release everything if job is finished
cap.release()
cv2.destroyAllWindows()

# Convert all_gestures to a NumPy array for further processing
all_gestures_np = np.array(all_gestures)
print("Data collection complete. Shape of collected data:", all_gestures_np.shape)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Gesture recorded. Total recorded gestures: 1
Gesture recorded. Total recorded gestures: 2
Gesture recorded. Total recorded gestures: 3
Gesture recorded. Total recorded gestures: 4
Gesture recorded. Total recorded gestures: 5
Gesture recorded. Total recorded gestures: 6
Gesture recorded. Total recorded gestures: 7
Gesture recorded. Total recorded gestures: 8
Gesture recorded. Total recorded gestures: 9
Gesture recorded. Total recorded gestures: 10
Gesture recorded. Total recorded gestures: 11
Gesture recorded. Total recorded gestures: 12
Gesture recorded. Total recorded gestures: 13
Gesture recorded. Total recorded gestures: 14
Gesture recorded. Total recorded gestures: 15
Gesture recorded. Total recorded gestures: 16
Gesture recorded. Total recorded gestures: 17
Gesture recorded. Total recorded gestures: 18
Gesture recorded. Total recorded gestures: 19
Gesture recorded. Total recorded gestures: 20
Data collection complete. Shape of collected data: (20,)


/var/folders/8v/1qfmkv8d6sj5j4v15xj2dmy40000gn/T/ipykernel_65977/105310586.py:76: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  all_gestures_np = np.array(all_gestures)


### Converting into Pickle Format

In [4]:
num = len(all_gestures)
label_list = [1] * num 
pose_list = all_gestures

dictionary = {
    "label": label_list,
    "pose": pose_list
}

with open("party_data.pickle", "wb") as f:
    pickle.dump(dictionary, f)